In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd
import numpy as np

## Crawling data

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd

class Boxoffice_of_Year():
    def __init__(self, obj_id, year):
        self.obj_id = obj_id
        self.year = year # Boxoffice year
        self.html = None
        self.title = []
        self.ids = []
        self.release_year = [] # Movie released year
        self.release_month = [] # Movie released month
        self.detail_html = [] # Detail page for each movie
        self.nation = [] # Countries
        self.genre = [] # Genres
        self.netizen_grade = []  # Netizen grades
        self.rating = [] # Ratings
        self.total_audience = [] # Total audience
    
    def get_html(self):
        # Access the yearly box office page by url
        url = "https://movie.daum.net/boxoffice/yearly?year=" + self.year
        # Add user-agent header to prevent block from the web site
        response = requests.get(url, headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'})
        if response.status_code == 200:
            print("Successfully requested")
        self.html = bs(response.text,'html.parser')
        return self

    def get_movie_id(self):
        movie_list = self.html.findAll('a',{'class':'name_movie'})
        # Movie ID is in <a> tag 'href'. Movie ID is used for detail page
        for movie in movie_list:
            if (len(movie['href'][22:])) > 1:
                self.ids.append(movie['href'][22:])
            else:
                self.ids.append("None")
        
        # Get detail page with movie id
        for i in self.ids:
            detail_url = 'https://movie.daum.net/moviedb/main?movieId=' + i
            response = requests.get(detail_url, headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'})
            tmp = bs(response.text,'html.parser')
            self.detail_html.append(tmp)
        return self

    # Get release date (Year and month)
    def get_date(self):
        date_list = self.html.findAll('span',{'class':'info_state'})
        for i in date_list:
            self.release_year.append('20' + i.text.split()[0].split('.')[0])
            self.release_month.append(i.text.split()[0].split('.')[1])
        return self

    def get_title(self):
        movie_list = self.html.findAll('a',{'class':'name_movie'})
        for movie in movie_list:
            self.title.append(movie.text)
        return self

    def get_grade(self):
        # 한 해의 50개의 영화를 한 번에 find, 각 iteration마다 영화 하나의 점수, 평론가점수는 19년도부터 없어서 제외
        wrap_grade = self.html.findAll('span', {'class':'wrap_grade grade_netizen'})
        for i in wrap_grade:
            num_grade = i.findAll('span', {'class':'num_grade'})
            tmpnum = ''
            for j in num_grade:
                if j.text.isdigit():
                    tmpnum += str(int(j.text))
                else:
                    tmpnum += '.'
            self.netizen_grade.append(tmpnum)
        return self

    def get_audience(self):
        # Link to detail page of each movie by movie id and get audience by json
        for i in self.ids:
            detail_page = 'https://movie.daum.net/main/totalAudience.json?movieId=' + i
            try:
                parser = requests.get(detail_page).text
                dic = json.loads(parser)
                total_aud = dic['totalAudience']   
            except:
                total_aud = 'None'
            self.total_audience.append(total_aud)
        return self

    def get_rating(self):
        rating_list = self.html.findAll('em', {'class': 'ico_movie'})
        for i in rating_list:
            self.rating.append(i.text)
        return self

    
    def get_nation(self):
        for i in self.detail_html:
            if i.select_one("dl.list_main dd:nth-of-type(2)") is not None:
                each_nation = i.select_one("dl.list_main dd:nth-of-type(2)").text
            else:
                each_nation = "None"
            # each_nation = i.select_one("dl.list_main dd:nth-of-type(2)")            
            self.nation.append(each_nation)
        return self

    def get_genre(self):
        for i in self.detail_html:
            if i.select_one("dl.list_main dd:nth-of-type(1)") is not None:
                each_genre = i.select_one("dl.list_main dd:nth-of-type(1)").text
            else:
                each_genre = "None"
            self.genre.append(each_genre)
        return self

    def get_data(self):
        # 연도, 제목, 평점, 누적 관객수만을 빼고 삭제
        movie_data = self.__dict__.copy()
        del_li = []
        del_li = ['html','ids','obj_id', 'detail_html']
        for i in del_li:
            del movie_data[i]
        print('Successfully get ' + self.year + ' data.')
        return movie_data

years_ = ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
movie_obj = []
df = pd.DataFrame()
for i, j in enumerate(years_):
    obj = Boxoffice_of_Year(i,j)
    movie_obj.append(obj)
    obj.get_html()
    obj.get_movie_id()
    obj.get_title()
    obj.get_grade()
    obj.get_audience()
    obj.get_rating()
    obj.get_genre()
    obj.get_nation()
    obj.get_date()
    movie_data = obj.get_data()
    movie_data = pd.DataFrame(movie_data,index=obj.ids)
    df = pd.concat([df, movie_data])      

Successfully requested
Successfully get 2004 data.
Successfully requested
Successfully get 2005 data.
Successfully requested
Successfully get 2006 data.
Successfully requested
Successfully get 2007 data.
Successfully requested
Successfully get 2008 data.
Successfully requested
Successfully get 2009 data.
Successfully requested
Successfully get 2010 data.
Successfully requested
Successfully get 2011 data.
Successfully requested
Successfully get 2012 data.
Successfully requested
Successfully get 2013 data.
Successfully requested
Successfully get 2014 data.
Successfully requested
Successfully get 2015 data.
Successfully requested
Successfully get 2016 data.
Successfully requested
Successfully get 2017 data.
Successfully requested
Successfully get 2018 data.
Successfully requested
Successfully get 2019 data.
Successfully requested
Successfully get 2020 data.


In [4]:
df.head()

,year,title,release_year,release_month,nation,genre,netizen_grade,rating,total_audience
36635,2004,태극기 휘날리며,2004,02,한국,전쟁,9.1,15세관람가,"11,746,135"
37066,2004,트로이,2004,05,"미국, 몰타, 영국",액션/로맨스/멜로,8.4,15세관람가,"2,001,318"
39368,2004,내 머리 속의 지우개,2004,11,한국,로맨스/멜로/드라마,8.7,12세관람가,"1,887,868"
39411,2004,귀신이 산다,2004,09,한국,코미디/판타지/공포,6.4,12세관람가,"1,875,936"
38807,2004,투모로우,2004,06,미국,액션/스릴러/SF/어드벤처/드라마,8.2,12세관람가,"1,830,767"


In [5]:
%cd /content/drive/My Drive/Colab Notebooks/bdproj

/content/drive/My Drive/Colab Notebooks/bdproj


In [0]:
df.to_csv("data.csv", mode='w')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 850 entries, 36635 to 131556
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            850 non-null    object
 1   title           850 non-null    object
 2   release_year    850 non-null    object
 3   release_month   850 non-null    object
 4   nation          850 non-null    object
 5   genre           850 non-null    object
 6   netizen_grade   850 non-null    object
 7   rating          850 non-null    object
 8   total_audience  850 non-null    object
dtypes: object(9)
memory usage: 66.4+ KB
